In [4]:
import numpy as np

# ASCON S-box

In [5]:
S5 = (4, 11, 31, 20, 26, 21, 9, 2, 27, 5, 8, 18, 29, 3, 6, 28,
      30, 19, 7, 14, 0, 13, 17, 24, 16, 12, 1, 25, 22, 10, 15, 23)

In [6]:
def bin_array(num, m):
    """Convert a positive integer num into an m-bit bit vector"""
    return np.array(list(np.binary_repr(num).zfill(m))).astype(np.int8)

# Differential profile

In [27]:
ASCON_diff = np.zeros((32,32))

In [28]:
for input_diff in range(32):
    for i in range(32):
        output_diff = S5[i] ^ S5[i^input_diff]
        ASCON_diff[int(input_diff)][int(output_diff)] +=1

In [29]:
for i in range(32):
    print ("\n")
    for j in range(32):
        print (str(int(ASCON_diff[i][j])).rjust(2), end='')



32 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

 0 0 0 0 0 0 0 0 0 4 0 4 0 4 0 4 0 0 0 0 0 0 0 0 4 0 4 0 4 0 4 0

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 4 0 4 0 4 0 4 0 4 0 4 0 4

 0 4 0 0 0 4 0 0 0 4 0 0 0 4 0 0 4 0 0 0 4 0 0 0 4 0 0 0 4 0 0 0

 0 0 0 0 0 0 8 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0 8 0

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 4 4 0 4 0 4 0 4 0 0 4 0 4

 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2

 0 0 4 4 0 0 4 4 0 0 4 4 0 0 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

 0 0 0 0 0 0 4 4 0 0 0 0 0 0 4 4 0 0 0 0 0 0 4 4 0 0 0 0 0 0 4 4

 0 2 0 2 2 0 2 0 2 0 2 0 0 2 0 2 2 0 2 0 0 2 0 2 0 2 0 2 2 0 2 0

 0 2 2 0 2 0 0 2 0 2 2 0 2 0 0 2 0 2 2 0 2 0 0 2 0 2 2 0 2 0 0 2

 0 0 2 2 0 0 2 2 0 0 2 2 0 0 2 2 0 0 2 2 0 0 2 2 0 0 2 2 0 0 2 2

 0 8 0 0 0 0 0 0 8 0 0 0 0 0 0 0 8 0 0 0 0 0 0 0 0 8 0 0 0 0 0 0

 0 2 0 2 0 2 0 2 2 0 2 0 2 0 2 0 2 0 2 0 2 0 2 0 0 2 0 2 0 2 0 2

 0 4 4 0 4 0 0 4 0 0 0 0 0 0 0 0 0 4 4 0 4 0 0 4 0 0 0 0 0 0 0 0

 0 0 0 0

# Linear profile

In [7]:
ASCON_linear = np.zeros((32,32))

In [8]:


for i in range(32):
    for input_mask in range(1,32):
        for output_mask in range(1,32):
            v1 = bin_array(i, 7)
            v2 = bin_array(S5[i], 7)
            sp1 = np.dot(v1,bin_array(input_mask, 7).T)%2
            sp2 = np.dot(v2,bin_array(output_mask, 7).T)%2
            if (sp1==sp2): ASCON_linear[input_mask][output_mask] += 1
            if (sp1!=sp2): ASCON_linear[input_mask][output_mask] -= 1    

In [26]:
np.max(np.abs(ASCON_linear)) #max absolute value in the linear profile

16.0

In [24]:
for i in range(32):
    print ("\n")
    for j in range(32):
        print (str(int(ASCON_linear[i][j])).rjust(3), end='')



  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

  0  0  0  0  0  0 16  0  0  8  8  0  0 -8  8  0  0  0  8  8  0  0  8 -8  8  0 -8  0 -8  0 -8  0

  0  0  0  0  0  0-16 16  0  0  8  8  0  0  8  8  0  0  8  8  0  0 -8 -8  0  0  0  0  0  0  0  0

  0 16  0  0  0  0  0  0  0  8  0  8  0  8  0 -8-16  0  0  0  0  0  0  0  8  0  8  0  8  0 -8  0

  0  0  0  8  0 -8  0  0  0  0  8  0  0  8 -8 -8  0  0  8  0 -8  0  0  0  0-16  0 -8 -8  0  8 -8

  0  0  0  8  0  8  0  0  0 -8  0  0  0  0  0 -8  0  0  0 -8  8  0 -8 -8  8  0 -8  8  0-16  0 -8

  0  0  0  8  0 -8  0  0  0  0  0 -8  0  8  0  0  0  0  0 -8 -8  0 -8 -8  0 16  0 -8 -8  0 -8  8

  0  0  0 -8  0 -8  0  0  0  8  8  8  0  0 -8  0  0  0 -8  0 -8  0  0  0 -8  0 -8  8  0-16  0  8

  0  0  0  0  0  0  0  0  0  0  8  8  0  0 -8 -8  0  0  0  0  0  0  0  0  0 16 -8  8  0 16  8 -8

  0  0  0  0  0  0  0-16  0 -8  0  8  0  8  0  8  0  0  8  8  0  0 -8  8  8  0  0  8 -8  0  0  8

  0  0  0  0  0  0